# Kahoot Ranking

Este Notebook filtra os vencedores do Kahoot, retornando os primeiros n colocados.

## ▶ Pontuação
### 1. Pontos de pódio
Para cada Kahoot:

🥇1 lugar : 3 pontos

🥈2 lugar : 2 pontos

🥉3 lugar : 1 ponto

### 2. Pontos do Kahoot
O desempate é feito pela pontuação acumulada dos Kahoots. 

In [1]:
import os
import pandas as pd
from unidecode import unidecode
import re

In [2]:
# Set folder path
folder_path = "2024_1-LIBERDADE"

In [3]:
def clean_string(input_string):
    # Use a regular expression to remove spaces and numbers
    cleaned_string = re.sub(r'[\s\d]', '', input_string)

    # Remove accents using unidecode
    no_accents_string = unidecode(cleaned_string)

    # Convert the cleaned string to lowercase
    lowercased_string = no_accents_string.lower()

    return lowercased_string

## Name exception dict 
Define name aliases to substitute (check which names have more than 1 alias)

'alias': 'originalname'

In [10]:
name_alias = {
    'marbinoso': 'mardabi', 
    'bine': 'bidomar', 
    'mardomar': 'johann',
    'quadra': 'johann',
    'quadrado': 'johann',
    'bigxand': 'xandao',
    'shaquilleomeal': 'johann',
    'bobberkurwa': 'johann',
    'paracetamal': 'johann',
    'quadroh': 'johann',
    'vaixco ': 'johann',
    'bagriel': 'johann',
    'mardomar': 'johann',
    'quadrado': 'johann'
}

In [12]:
# Get file info
file_list = [f for f in os.listdir(folder_path)]

df_dict = {}

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    key = file_name
    df_dict[key] = pd.read_excel(file_path, sheet_name='Final Scores')

# Create dataframe for podium
general_podium = pd.DataFrame()

# Join the files
for key in df_dict:
    podium = df_dict[key].iloc[2:5,:3].rename(columns={df_dict[key].columns[0]:'Podium'})
    general_podium = pd.concat([general_podium,podium])

# Rename columns
general_podium.rename(columns={'Unnamed: 1':'Name','Unnamed: 2':'Points'},inplace=True)
# Change data type
general_podium['Podium'] = general_podium['Podium'].astype(int)
general_podium['Points'] = general_podium['Points'].astype(int)

# Clean names
general_podium['Name'] = general_podium['Name'].apply(lambda x: clean_string(x))

# Substitute name aliases
general_podium['Name'] = general_podium['Name'].replace(name_alias)

# Assign Podium points
point_mapping = {1: 3, 2: 2, 3: 1}
general_podium['Podium_Points'] = general_podium['Podium'].map(point_mapping)

# Create general podium
general_podium = (general_podium.loc[:,['Name','Podium_Points','Points']]
                  .groupby(['Name'])
                  .sum()
                  .sort_values(['Podium_Points', 'Points'], ascending=[False, False])
                  .reset_index())

# Index starts at 1
general_podium.index = general_podium.index + 1

general_podium.head(5)

,Name,Podium_Points,Points
1,johann,12,31247
2,natal,10,23765
3,einstein,8,16101
4,lu,6,13104
5,marcell,5,11199
